In [1]:
#!pip install langchain_community
#!pip install chromadb
#!pip install openai

In [2]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
import os
import json
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import time
import pandas as pd

In [3]:
# 设置OpenAI API密钥
f = open("./openai_key.txt", 'r')
openai_key=f.read()
f.close()
os.environ["OPENAI_API_KEY"] = openai_key

# 加载CSV文件
csv_file_path = './ptt_stock_articles.csv'
comments_file_path = './ptt_stock_comments.csv'

df = pd.read_csv(csv_file_path)
df_comments = pd.read_csv(comments_file_path)

# 将CSV文件转换为JSONL文件
jsonl_file_path = 'combined_ptt_stock_documents.jsonl'
df.to_json(jsonl_file_path, orient='records', lines=True, force_ascii=False)

In [4]:
df

,URL,Title,Content
0,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,"[新聞] 由 4,300 公里海底電纜從澳洲輸電新加坡","原文標題：\n\n由 4,300 公里海底電纜從澳洲輸電新加坡，計畫獲重要進展\n\n原文連..."
1,https://www.ptt.cc/bbs/Stock/M.1721364406.A.4F...,[新聞] 不受川普保護費說影響 中經院1理由上修台,原文標題：不受川普保護費說影響 中經院1理由上修台灣經濟成長率\n\n\n原文連結：\n\n...
2,https://www.ptt.cc/bbs/Stock/M.1721365467.A.4E...,[情報] 2429 銘旺科 申購抽籤日程資訊,1. 標題：2429 銘旺科 申購抽籤日程資訊\n\n2. 來源：TWSE 公開申購公告-抽...
3,https://www.ptt.cc/bbs/Stock/M.1721365814.A.7A...,[心得] 台積電是咒術迴戰裡面的真人,台積電 --> 真人\n\n美國 --> 夏油傑\n\n美國根本就是牢牢的掌握台積電，\n等...
4,https://www.ptt.cc/bbs/Stock/M.1719872231.A.9B...,[公告] 股票板板規 v4.6 (2024/07/02 修正),1. 發文規範與分類總則 2. 股票板板規執法範圍 --------------------...
5,https://www.ptt.cc/bbs/Stock/M.1721349002.A.3F...,[閒聊] 2024/07/19 盤中閒聊,==============113/07/19台股資訊重點整理，供股民做投資參考======...
6,https://www.ptt.cc/bbs/Stock/M.1721356359.A.52...,[請益] 請問華航怎麼了？,一堆利好跟利多\nMSCI空運物流指數也持續在上升\n華航可以跌倒跌成這樣\n這波根本完全都...
7,https://www.ptt.cc/bbs/Stock/M.1721356684.A.29...,[標的] 6869 雲豹能源 空,標的：6869 雲豹能源\n\n\n分類：空\n\n\n分析/正文：\n\n看到今天這根漲停...
8,https://www.ptt.cc/bbs/Stock/M.1721357100.A.14...,Re: [情報] 6405悅城處分台積電普通股股票之公告,中環在幾天前高割離席處分台積電股票，反觀悅城破千才要進來，上次也是套在最高點，之後血虧出場，...
9,https://www.ptt.cc/bbs/Stock/M.1721357144.A.38...,[新聞] 高盛：人工智慧投資正在泡沫化、短期內,高盛：人工智慧投資正在泡沫化、短期內還不會破裂\n\n\n\nMoneyDJ新聞 2024-...


In [5]:
df_comments

,URL,push_tag,push_userid,push_content,push_time
0,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,→,Fezico,這真的很狂...,07/19 12:45
1,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,推,cchh179,這麼長要消耗多少,07/19 12:46
2,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,→,KJK7,漁船應該很容易勾斷,07/19 12:47
3,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,推,oyaji5566,[新聞] 海底「松鼠」出沒,07/19 12:47
4,https://www.ptt.cc/bbs/Stock/M.1721364282.A.AC...,推,a069275235,笑死 厲害了綠能仔,07/19 12:47
...,...,...,...,...,...
2927,https://www.ptt.cc/bbs/Stock/M.1721363445.A.44...,→,v21638245,都還行啊,07/19 12:42
2928,https://www.ptt.cc/bbs/Stock/M.1721363445.A.44...,推,joshua1226,党,07/19 12:57
2929,https://www.ptt.cc/bbs/Stock/M.1721363445.A.44...,推,fungling,合一 三期 大陸藥證 幫哭哭,07/19 12:59
2930,https://www.ptt.cc/bbs/Stock/M.1721363445.A.44...,推,sharkman1793,00713也可以,07/19 13:01


In [6]:
data_all_list = []
for i in range(df.shape[0]):
    data_temp = pd.DataFrame({"Content":[df["Content"][i]], "push_content":[str(df_comments[df_comments["URL"]==df["URL"][i]]["push_content"].to_list())]})
    data_all_list.append(data_temp)
data_all = pd.concat(data_all_list).reset_index(drop=True)

In [7]:
for_langchain_path = "./for_langchain_data.csv"
data_all.to_csv(for_langchain_path)

In [8]:
# # 加载JSONL文件
# loader = JSONLoader(
#     file_path=jsonl_file_path,
#     jq_schema='.Content',
#     json_lines=True,
#     text_content=True
# )

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [10]:
# articles_loader = CSVLoader(file_path=csv_file_path,encoding="utf-8")
# articles_data = articles_loader.load()
# comments_loader = CSVLoader(file_path=comments_file_path,encoding="utf-8")
# comments_data = comments_loader.load()
# documents_load = articles_data + comments_data
loader = CSVLoader(file_path=for_langchain_path,encoding="utf-8")
documents_load = loader.load()

In [11]:
# 初始化OpenAI Embeddings
embeddings = OpenAIEmbeddings()

C:\Users\User\anaconda3\envs\py310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents_load)

In [13]:
# from langchain.embeddings import HuggingFaceEmbeddings
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# model_kwargs = {'device': 'cpu'}
# embedding = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)

In [14]:
%%time
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory=persist_directory)

CPU times: total: 1.84 s
Wall time: 4.38 s


In [15]:
doc_search_result = vectordb.similarity_search_with_score("")

In [16]:
doc_search_result[0]

(Document(metadata={'row': 22, 'source': './for_langchain_data.csv'}, page_content='--'),
 0.5442936369131246)

In [21]:
sys_prompt = f"""你在Thought、Action、PAUSE、Observation的循環中運行。
在循環結束時，你輸出Answer
使用Thought描述你對被問問題的想法
使用Action執行你可以使用的行動之一，然後返為PAUSE。
Observation將是執行這些行動的結果。

Thought: 你是一個銀行行員
Action: 請用銀行行員的角度來看以下文章的情緒
Observation:銀行行員的情緒分數

Thought: 你是一個酸民
Action: 請用酸民的角度來看以下文章的情緒
Observation:酸民的情緒分數

Thought: 你是一個路人
Action: 請用路人的角度來看以下文章的情緒
Observation:路人的情緒分數

Thought: 你是一個老師
Action: 請用老師的角度來看以下文章的情緒
Observation:老師的情緒分數

Thought: 你是一個資深股民
Action: 請用資深股民的角度來看以下文章的情緒
Observation:資深股民的情緒分數

Thought: 你需要統整以上的資訊
Action: 請用統整以上的資訊並給出情緒
Observation:統整出來的情緒分數要介於-10到10之間的數值，請檢察回

"""

In [77]:
# 進行檢索與生成
from langchain import hub
retriever = vectordb.as_retriever(search_page="similarity", search_kwargs={"k": 10})
prompt = hub.pull("rlm/rag-prompt")
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
#llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [118]:
prompt.messages[0].prompt.template = """
你在Thought、Action、PAUSE、Observation的循環中運行。
在循環結束時，你輸出Answer
使用Thought描述你對被問問題的想法
使用Action執行你可以使用的行動之一，然後返為PAUSE。
Observation將是執行這些行動的結果。

Thought: 你是一個銀行行員
Action: 請用銀行行員的角度來看以下文章的情緒
Observation:銀行行員的情緒分數

Thought: 你是一個酸民
Action: 請用酸民的角度來看以下文章的情緒
Observation:酸民的情緒分數

Thought: 你是一個路人
Action: 請用路人的角度來看以下文章的情緒
Observation:路人的情緒分數

Thought: 你是一個老師
Action: 請用老師的角度來看以下文章的情緒
Observation:老師的情緒分數

Thought: 你是一個資深股民
Action: 請用資深股民的角度來看以下文章的情緒
Observation:資深股民的情緒分數

Thought: 你需要統整以上的資訊
Action: 請用統整以上的資訊並給出情緒
Observation:統整出來的情緒分數要介於-10到10之間的數值，請檢察回覆內容

Question: {question} \nContext: {context} \nAnswer:
"""

In [121]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
def format_docs(docs):
    for doc in docs:
        print(doc.page_content)
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("完成建立RAG")

result = rag_chain.invoke(f"""台積電的情緒如何?""")

完成建立RAG
: 16
Content: 基於各種原因
最近川普的發言似乎對台積電很不利
也導致了台股及台積電下跌
那如果這個時候
川普購入了台積電股票
算是內線交易
或操縱股票嗎？
: 22
Content: 原文標題：

川普口水有多毒？曝光台積電最大弱點 命運被2國擺弄


原文連結：



發布時間：
05:002024/07/19 中時新聞網 吳美觀

記者署名：
05:002024/07/19 中時新聞網 吳美觀


原文內容：



美國總統候選人川普16日表示，，還說要向台灣收保護費，此
話一出，立馬衝擊台股和台積電股價。外媒報導，台積電雖居全球半導體領先地位，但川
普的無情發言也凸顯台積電的脆弱，其最大弱點就是命運受到北京與華盛頓影響。

路透報導，現任拜登政府明確地表達對台灣的支持，還說，不排除在中國入侵時，美國軍
事力量介入。川普則批評台灣不僅搶走美國的晶片事業，且台灣沒有給美國任何回報，應
該要付出保護費。

對於台積電這家市值高達8400億美元（約台幣27兆元）的半導體巨頭而言，川普的話殘酷
地提醒，其命運可能受制於北京與華盛頓。台積電17日股價重挫近3%，市值蒸發230億美
元（約台幣7400億元）。

從今年5月以來，幾乎每天都有大陸軍機在台灣附近執行任務。難怪川普此言一出，台積
電股價立即下挫，在AI熱潮帶來榮景之際，投資人擔憂不斷惡化的美中關係，成為市場的
巨大風險。

報導指出，即使把川普的「保護費說」視為選舉語言，但他重申加徵關稅的態度及發放補
貼的不確定性，應該會讓在美國設廠的台積電感到不安。

分析師預估，到2025年，台積電在AI相關收入將成長4倍，占其營收五分之一；台積電今
年股價雖然大漲，平均本益比較費半指數其他公司來得低，投資價值相對穩健。


心得/評論：

1. 川普說台灣搶走美國所有的晶片生意，現在台灣錢很多，我們應該從台灣的口袋
   掏錢。

2. 台積電本益比相較費半指數其他公司仍然較低，具有投資價值
'官田鋼跟台達電合作儲能槽，可以算綠電概念股嗎', '在幻想嗎？還沒有出來融資維持率不足的新聞', '媽的 樓上貼恐怖圖 我要黑你', '月線撿發哥會贏嗎', '冷門阿貓阿狗沒殺意正常 量那麼少只有特定人士在那', '喊盤而已', '降息循環平均跌43%，多蛙還在樂觀', 'GG越跌越買  4月法會後的走

In [122]:
print(result)

Thought: 你是一個銀行行員
Action: 請用銀行行員的角度來看以下文章的情緒
Observation: 銀行行員的情緒分數

Thought: 你是一個酸民
Action: 請用酸民的角度來看以下文章的情緒
Observation: 酸民的情緒分數

Thought: 你是一個路人
Action: 請用路人的角度來看以下文章的情緒
Observation: 路人的情緒分數

Thought: 你是一個老師
Action: 請用老師的角度來看以下文章的情緒
Observation: 老師的情緒分數

Thought: 你是一個資深股民
Action: 請用資深股民的角度來看以下文章的情緒
Observation: 資深股民的情緒分數

Thought: 你需要統整以上的資訊
Action: 請用統整以上的資訊並給出情緒
Observation: 統整出來的情緒分數要介於-10到10之間的數值，請檢察回覆內容

---

Observation: 銀行行員的情緒分數：-5
Observation: 酸民的情緒分數：-8
Observation: 路人的情緒分數：-3
Observation: 老師的情緒分數：-4
Observation: 資深股民的情緒分數：-7

Thought: 綜合以上不同角色的情緒分數，台積電的情緒整體偏負面，主要受到川普發言和台股下跌的影響。各角色的情緒分數均在-3到-8之間，顯示出對台積電未來的不確定性和擔憂。

Action: 統整出來的情緒分數要介於-10到10之間的數值，請檢察回覆內容
Observation: 統整出來的情緒分數為-5.4

Answer: 台積電的情緒分數為-5.4，顯示出市場對其未來前景的擔憂和不確定性。


In [80]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [81]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever(search_page="similarity", search_kwargs={"k": 10})
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [86]:
question = f"""你在Thought、Action、PAUSE、Observation的循環中運行。
在循環結束時，你輸出Answer
使用Thought描述你對被問問題的想法
使用Action執行你可以使用的行動之一，然後返為PAUSE。
Observation將是執行這些行動的結果。

Thought: 你是一個銀行行員
Action: 請用銀行行員的角度來看以下文章的情緒
Observation:銀行行員的情緒分數

Thought: 你是一個酸民
Action: 請用酸民的角度來看以下文章的情緒
Observation:酸民的情緒分數

Thought: 你是一個路人
Action: 請用路人的角度來看以下文章的情緒
Observation:路人的情緒分數

Thought: 你是一個老師
Action: 請用老師的角度來看以下文章的情緒
Observation:老師的情緒分數

Thought: 你是一個資深股民
Action: 請用資深股民的角度來看以下文章的情緒
Observation:資深股民的情緒分數

Thought: 你需要統整以上的資訊
Action: 請用統整以上的資訊並給出情緒
Observation:統整出來的情緒分數要介於-10到10之間的數值，請檢察回

"""
result = qa({"question": question})

In [87]:
print(result["answer"])

### Thought
我需要從銀行行員、酸民、路人、老師和資深股民的角度來分析這篇文章的情緒，並給出情緒分數。這篇文章主要討論了股市的波動、投資者的反應以及對未來市場的預測。情緒分數將基於這些角度的觀點來評估。

### Action
1. **銀行行員的角度**：銀行行員通常會關注市場的穩定性和客戶的投資情緒。
2. **酸民的角度**：酸民通常會有較為負面的評論，並且喜歡挑剔和批評。
3. **路人的角度**：路人可能對股市沒有太多深入了解，情緒反應會較為中立或隨大流。
4. **老師的角度**：老師會關注教育和知識傳遞，可能會對市場的波動有理性的分析。
5. **資深股民的角度**：資深股民會有豐富的市場經驗，對市場波動有較為冷靜和專業的看法。

### PAUSE
我將從這些角度來分析文章的情緒。

### Observation
1. **銀行行員的角度**：
   - 文章中提到市場的波動和投資者的反應，銀行行員可能會擔心客戶的投資情緒和資金流動。
   - 情緒分數：-1（略微負面）

2. **酸民的角度**：
   - 文章中有不少負面評論和對市場的批評，酸民會覺得這些內容符合他們的觀點。
   - 情緒分數：-2（負面）

3. **路人的角度**：
   - 路人可能會覺得市場波動是正常現象，情緒反應較為中立。
   - 情緒分數：0（中立）

4. **老師的角度**：
   - 老師會關注市場波動的原因和教育意義，可能會對市場有理性的分析。
   - 情緒分數：+1（略微正面）

5. **資深股民的角度**：
   - 資深股民會有豐富的市場經驗，對市場波動有較為冷靜和專業的看法，可能會看到投資機會。
   - 情緒分數：+1（略微正面）

### Answer
綜合以上分析，文章的情緒分數如下：
- 銀行行員：-1
- 酸民：-2
- 路人：0
- 老師：+1
- 資深股民：+1

總體情緒分數：(-1) + (-2) + 0 + (+1) + (+1) = -1

因此，這篇文章的總體情緒分數為 -1，表示情緒略微負面。


In [20]:
# from langchain_community.chat_models import ChatOpenAI
# from langchain.chains import ConversationalRetrievalChain
# import os
# import json
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
# from langchain_community.document_loaders import JSONLoader
# import chromadb
# from chromadb.config import Settings
# from chromadb.utils import embedding_functions
# import time
# import pandas as pd

# # 设置OpenAI API密钥
# os.environ["OPENAI_API_KEY"] = ""

# # 加载CSV文件
# csv_file_path = 'ptt_stock_articles.csv'
# comments_file_path = 'ptt_stock_comments.csv'

# df = pd.read_csv(csv_file_path)
# df_comments = pd.read_csv(comments_file_path)

# # 将CSV文件转换为JSONL文件
# jsonl_file_path = 'combined_ptt_stock_documents.jsonl'
# df.to_json(jsonl_file_path, orient='records', lines=True, force_ascii=False)

# # 加载JSONL文件
# loader = JSONLoader(
#     file_path=jsonl_file_path,
#     jq_schema='.Content',
#     json_lines=True,
#     text_content=True
# )

# # 初始化OpenAI Embeddings
# embeddings = OpenAIEmbeddings()

# # 加载文档
# documents_load = loader.load()

# # 初始化Chroma并将文档添加到Chroma
# chroma_client = chromadb.Client(Settings())

# # 检查集合是否存在
# collection_name = 'recommend_collection'
# collection_list = chroma_client.list_collections()

# if collection_name in [col.name for col in collection_list]:
#     print(f"Collection {collection_name} already exists. Deleting the existing collection.")
#     chroma_client.delete_collection(collection_name)

# # 创建集合
# collection = chroma_client.create_collection(
#     name=collection_name,
#     embedding_function=embedding_functions.OpenAIEmbeddingFunction(
#         api_key=os.environ['OPENAI_API_KEY'],
#         model_name="text-embedding-ada-002"
#     )
# )

# # 处理文档并将它们添加到Chroma
# def process_documents_and_add_to_chroma(documents, collection, batch_size=100, delay=60):
#     total_docs = len(documents)
#     for start_idx in range(0, total_docs, batch_size):
#         end_idx = min(start_idx + batch_size, total_docs)
#         batch = documents[start_idx:end_idx]
#         document_ids = [f"doc_{i}" for i in range(start_idx, end_idx)]  # 生成文档ID列表
#         batch_contents = [doc.page_content[:1000] for doc in batch]  # 提取文档内容并限制长度为1000字符
        
#         try:
#             # 使用 embed_documents 方法来获取批量文档的嵌入
#             batch_embeddings = embeddings.embed_documents(batch_contents)
            
#             # 确保嵌入和ID数量一致
#             if len(batch_embeddings) != len(batch_contents):
#                 raise ValueError(f"Number of embeddings ({len(batch_embeddings)}) must match number of documents ({len(batch_contents)})")
            
#             # 确保 ID 数量与文档数量一致
#             if len(document_ids) != len(batch_contents):
#                 raise ValueError(f"Number of document IDs ({len(document_ids)}) must match number of documents ({len(batch_contents)})")
            
#             collection.add(documents=batch_contents, embeddings=batch_embeddings, ids=document_ids)
            
#         except Exception as e:
#             print(f"Error processing batch from index {start_idx} to {end_idx}: {e}")
        
#         if end_idx < total_docs:
#             print(f"Batch {start_idx//batch_size + 1} completed. Waiting for {delay} seconds...")
#             time.sleep(delay)

# # 处理文档并将它们添加到Chroma
# process_documents_and_add_to_chroma(documents_load, collection, batch_size=100, delay=60)

# # 定义新的股票情绪分析prompt模板
# from langchain.prompts import (
#     ChatPromptTemplate,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )
# query = "最近台股情绪如何？"
# user_question = query

# template = """你是一個股票分析助手，根據使用者提供的問題以及PTT上的文章和留言進行股票情緒分析。

# PTT上的文章和留言: {background_context}

# Hard requirements and steps that must be followed:
# 1. 請描述使用者提供的問題。
# 2. 使用提供的PTT資料，分析台股情绪。
# 3. 請列出文章標題和主要內容。
# 4. 請總結每篇文章的情绪（正面、負面、中立）。
# 5. 如果有不一致的評價，請詳細說明。
# 6. 請使用簡明扼要的語言回答。

# 用詞要有禮貌與專業！"""

# system_message_prompt = SystemMessagePromptTemplate.from_template(template)
# human_template = "{user_question}"
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # 初始化ChatOpenAI
# chat_model = ChatOpenAI(model_name="gpt-4", temperature=0.7)

# # 遍历所有文档的标题和内容，并生成回应
# try:
#     responses = []
    
#     for doc in documents_load:
#         title = doc.metadata.get('title', 'No Title')  # 确保文档中有标题
#         content = doc.page_content[:1000]  # 限制内容长度为1000字符
        
#         # 生成回应
#         response = chat_model(chat_prompt.format_messages(background_context=[content], user_question=user_question))
#         responses.append(f"Title: {title}\nContent: {content}\nResponse: {response.content}\n\n")
    
#     # 将所有回应保存到文本文件
#     with open("generated_responses.txt", "w", encoding="utf-8") as f:
#         f.writelines(responses)
        
# except Exception as e:
#     response = f"Error during document processing or response generation: {e}"
#     print(response)
#     with open("generated_responses.txt", "w", encoding="utf-8") as f:
#         f.write(response)
    
# print(response)